# Remove PII from Dataverse Contacts

This notebook demonstrates using AI to remove PII (Personally Identifiable Information) from Dataverse contact records.

In [ ]:
from azure.identity import InteractiveBrowserCredential
from PowerPlatform.Dataverse.client import DataverseClient

In [ ]:
# Connect to Dataverse
credential = InteractiveBrowserCredential()
client = DataverseClient("https://ecellorsdev.crm8.dynamics.com", credential)

In [ ]:
# Function to remove PII data from contact records
def remove_pii_from_contact(contact):
    pii_fields = ['emailaddress1', 'telephone1', 'mobilephone', 'address1_line1', 'address1_city', 'address1_postalcode']
    for field in pii_fields:
        if field in contact:
            contact[field] = '[REDACTED]'
    return contact

In [ ]:
# Fetch contacts with PII (Dataverse client returns paged batches)
contact_batches = client.get(
    "contact",
    select=[
        "contactid",
        "fullname",
        "emailaddress1",
        "telephone1",
        "mobilephone",
        "address1_line1",
        "address1_city",
        "address1_postalcode",
    ],
    top=10,
)

In [ ]:
# Remove PII and update contacts
for batch in contact_batches:
    for contact in batch:
        contact_id = contact.get("contactid")
        sanitized_contact = remove_pii_from_contact(contact)

        # Prepare update data (exclude contactid)
        update_data = {key: value for key, value in sanitized_contact.items() if key != "contactid"}

        # Update the contact in Dataverse
        client.update("contact", contact_id, update_data)
        print(f"Contact {contact_id} updated with sanitized data: {sanitized_contact}")